In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import os


token_count = 30000


In [13]:
input_data = pd.read_pickle('../dados/df_ata_sentimento_predict.pkl') 
#input_data = pd.read_json('../dados/df_ata_sentimento.json', orient='records', lines=True)
#input_data['Data'] = pd.to_datetime(input_data['Data']).dt.date
input_data['Texto'] = input_data['Texto'].str.lower()

print(input_data)


          Data                                              Texto  Selic  \
234 2025-05-07  a atualização da conjuntura econômica e do cen...  14.25   
235 2025-06-18  a atualização da conjuntura econômica e do cen...  14.75   
236 2025-07-30  a atualização da conjuntura econômica e do cen...  15.00   
237 2025-09-17  a atualização da conjuntura econômica e do cen...  15.00   

     IPCA  Selic (6m) Data (6m)  
234  5.32         NaN       NaT  
235  5.35         NaN       NaT  
236  5.23         NaN       NaT  
237  5.17         NaN       NaT  


In [3]:
# Example: Recreate text_vec_layer if not saved
from tensorflow.keras.layers import TextVectorization

# You must know the vocabulary or adapt it again from training data
# If you saved the vocab, load it; otherwise, re-adapt on training data
text_vec_layer = TextVectorization(
    max_tokens=token_count,  # same as training
    output_mode='int'
)

# Re-adapt if needed (only on training texts!)
# text_vec_layer.adapt(train_texts)  # where train_texts is list of raw strings

In [6]:
# Load the best model saved via ModelCheckpoint
print(os.getcwd())
model = tf.keras.models.load_model('../../gru_sentimento_6m__GRU_0601_modl_sent_encoded_180_mzt_220_B32_VECLYR_.keras')


/Users/ricardo/Documents/Projeto2/Projeto-Aplicado2-Grupo19/Notebooks


In [22]:
this_text = input_data.iloc[0,1]
#print(this_text)
predictions = model.predict(this_text)
print(predictions)

ValueError: Unrecognized data type: x=a atualização da conjuntura econômica e do cenário do copom11 o ambiente externo mostrase adverso e particularmente incerto em função da conjuntura e da política econômica nos estados unidos principalmente acerca de sua política comercial e de seus efeitos2 a política comercial alimenta incertezas sobre a economia global notadamente acerca da magnitude da desaceleração econômica e sobre o efeito heterogêneo no cenário inflacionário entre os países com repercussões relevantes sobre a condução da política monetária3 além disso o comportamento e a volatilidade de diferentes classes de ativos também têm sido afetados com fortes reflexos nas condições financeiras globais tal cenário segue exigindo cautela por parte de países emergentes em ambiente de maior tensão geopolítica4 em relação ao cenário doméstico o conjunto dos indicadores de atividade econômica e do mercado de trabalho ainda tem apresentado dinamismo mas observase uma incipiente moderação no

In [7]:
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)
print(y_test)

NameError: name 'predictions' is not defined

In [ ]:
def preprocess_text(text, text_vec_layer):
    # Convert string to tensor
    text_tensor = tf.constant([text])
    # Vectorize
    vectorized = text_vec_layer(text_tensor)
    return vectorized

# Example input
new_text = "Your input string here"

# Preprocess
input_vectorized = preprocess_text(new_text, text_vec_layer)